In [12]:
import torch

## load checkpoint

In [18]:
pathe = '/home/rozenberszki/project/wsnsl/output/Own/room1/ckpts/01999.tar'
traj = '/home/rozenberszki/project/wsnsl/Datasets/Replica/room1/traj.txt'

In [19]:
d = torch.load(pathe)

In [20]:
T_e = d['estimate_c2w_list']
#T_e[:,1:3]*= -1

In [21]:
T_e[1600]

tensor([[-4.4940e-01, -2.9474e-01,  8.4331e-01, -1.6724e+00],
        [ 8.9333e-01, -1.5003e-01,  4.2362e-01,  1.8866e-01],
        [ 1.6611e-03,  9.4373e-01,  3.3072e-01,  7.7186e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [22]:
import numpy as np
T_wc = np.loadtxt(traj).reshape(-1,4,4)
T_wc[:,1:3]*= -1


In [28]:
np.round(T_e[5].numpy(),3)

array([[-0.11,  0.26,  0.96, -2.24],
       [ 0.99,  0.03,  0.11, -0.46],
       [ 0.  ,  0.96, -0.27,  0.19],
       [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32)

In [24]:
s = np.ones((4,4), int)
s[[0,0,1,1,2], [1,2,0,3,3]] *=-1

In [33]:
np.round(T_wc[10],2), np.round(T_e[10].numpy(),2),np.round(T_wc[5]- T_e[5].numpy(),3)

(array([[-0.24, -0.16, -0.96, -2.18],
        [-0.97,  0.04,  0.23,  0.43],
        [-0.  ,  0.99, -0.17, -0.21],
        [ 0.  ,  0.  ,  0.  ,  1.  ]]),
 array([[-0.24,  0.17,  0.95, -2.19],
        [ 0.97,  0.04,  0.24, -0.42],
        [-0.  ,  0.98, -0.18,  0.19],
        [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32),
 array([[-0.005, -0.436, -1.919,  0.002],
        [-1.968, -0.004,  0.006,  0.904],
        [ 0.002,  0.003,  0.015, -0.374],
        [ 0.   ,  0.   ,  0.   ,  0.   ]]))

In [26]:
T_e = T_e * s

In [32]:
torch.sum(T_e[50][:3,:3]**2)

tensor(3.0000, dtype=torch.float64)

In [34]:
T_e[50][:3,:3]

tensor([[-0.6137,  0.1284, -0.7791],
        [-0.7896, -0.0976,  0.6059],
        [-0.0017,  0.9869,  0.1613]], dtype=torch.float64)

## check decoder update

In [12]:
from src.conv_onet.models.decoder import DenseLayer
from src import config

In [13]:
cfg = config.load_config(  # J:changed it to use our config file including semantics
        '/home/koerner/Project/wsnsl/configs/Own/room1.yaml', "configs/nice_slam_sem.yaml"
    )

In [14]:

model = config.get_model(cfg, nice=True)


In [15]:
model

NICE(
  (middle_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (embedder): GaussianFourierFeatureTransform()
    (pts_linears): ModuleList(
      (0): DenseLayer(in_features=93, out_features=32, bias=True)
      (1): DenseLayer(in_features=32, out_features=32, bias=True)
      (2): DenseLayer(in_features=32, out_features=32, bias=True)
      (3): DenseLayer(in_features=125, out_features=32, bias=True)
      (4): DenseLayer(in_features=32, out_features=32, bias=True)
    )
    (output_linear): DenseLayer(in_features=32, out_features=1, bias=True)
  )
  (fine_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_fe

In [10]:
model.semantic_decoder.output_linear = DenseLayer(in_dim=32, out_dim=100, activation='linear')

In [11]:
model

NICE(
  (middle_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (embedder): GaussianFourierFeatureTransform()
    (pts_linears): ModuleList(
      (0): DenseLayer(in_features=93, out_features=32, bias=True)
      (1): DenseLayer(in_features=32, out_features=32, bias=True)
      (2): DenseLayer(in_features=32, out_features=32, bias=True)
      (3): DenseLayer(in_features=125, out_features=32, bias=True)
      (4): DenseLayer(in_features=32, out_features=32, bias=True)
    )
    (output_linear): DenseLayer(in_features=32, out_features=1, bias=True)
  )
  (fine_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_fe